# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ing. en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**: Real-time log analyzer

**Fecha**: 03 de abril del 2025

**Nombre del Estudiante**: Luis, Benja, Sam

**Profesor**: Pablo Camarillo Ramirez

In [122]:
import findspark
findspark.init()

In [123]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del FileStrean

In [ ]:
logs = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/logs/input/")

In [ ]:
from pyspark.sql.functions import split

logs_df = logs.select(split(logs.value, "|").alias("logs_array"))
logs_df = logs.withColumn("date", logs["logs_array"].getItem(0).cast("date"))
logs_df = logs.withColumn("error", logs["logs_array"].getItem(1).cast("string"))
logs_df = logs.withColumn("details", logs["logs_array"].getItem(2).cast("string"))
logs_df = logs.withColumn("server", logs["logs_aeeay"].getItem(3).cast("string"))


logs_df.printSchema()

In [ ]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='2 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)

In [121]:
sc.stop()